In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib
import torch
import matplotlib.pyplot as plt
# from tqdm import tqdm_notebook
%matplotlib inline 
# from google.colab.patches import cv2_imshow
from IPython.display import HTML #imports to play videos
from base64 import b64encode 
import cv2 as cv
#from skimage.measure import compare_ssim
from torchvision.datasets import ImageFolder
import glob
import time
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import math
import pickle
from functools import partial
from collections import defaultdict

from PIL import Image
from glob import glob
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import cv2
import skimage.measure
import albumentations as A
from tqdm.notebook import tqdm 
#from albumentations.pytorch import ToTensor 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models.video import mc3_18, r2plus1d_18

from facenet_pytorch import MTCNN



import sys
import sklearn

import cv2
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt
from torchvision import models, transforms
from torch.utils.data import WeightedRandomSampler

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

from torchsummary import summary
device = 'cuda'

DATA_FOLDER = os.getcwd()

BASE_PATH = os.getcwd()

ORI_CROPS = r'D:\FF\crops\original_sequences'
MANIP_CROPS = r'D:\FF\crops\manipulated_sequences'
CROPS_FOLDER = r'D:\FF\crops'
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.19 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model = EfficientNet.from_name('efficientnet-b1')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))
#model._fc = nn.Linear(in_features=model._fc.in_features, out_features=1)

class CustomModel(nn.Module):
    def __init__(self, pretrained):
        super(CustomModel, self).__init__()
        self.pretrained = pretrained  # Pre-trained model
        #self.global_avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(1280, out_features=512)  # Adjust output size as needed
        self.bn = nn.BatchNorm1d(num_features=512)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(in_features=512, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pretrained(x)  # Forward through the pre-trained model
        #x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)  # Flatten the output for the dense layers
        x = self.fc1(x)
        x = self.bn(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x).squeeze()
        return x
    

model = EfficientNet.from_pretrained(f"efficientnet-b0")
model._fc = nn.Identity()
for param in model.parameters():
    param.requires_grad = False
model = CustomModel(model)

Loaded pretrained weights for efficientnet-b0


In [3]:
summary(model, (32, 3, 224, 224))

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: [ZeroPad2d: 3-1]

In [5]:
class EarlyStopper:
    def __init__(self, patience=5, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = float('inf')  # Initialize to positive infinity
        self.verbose = verbose

    def early_stop(self, val_loss, model):
        if val_loss < self.best_score:
            self.best_score = val_loss  # Update best validation loss
            self.counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        else:
            self.counter += 1  # Increment patience counter
           
        if self.counter >= self.patience:
            if self.verbose:
                print("Early stopping...")
            return True
        return False

In [6]:
frames_per_video = 32
mtcnn = MTCNN(margin=20, select_largest=False, factor=0.5, device=device, post_process=True) # post_process=False if want human readable image
img_size = 224 # 256
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
class ImageTransform:
    def __init__(self, size, mean, std):
        self.data_transform = transforms.Compose([
                transforms.Resize((size, size), interpolation=Image.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])

    def __call__(self, img):
        return self.data_transform(img)
    
transformer = ImageTransform(img_size, mean, std)

c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

c:\Users\Sheryl\A

In [7]:


# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_=48, pin_memory=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []


In [8]:
train_dataset = ImageFolder(root=CROPS_FOLDER, transform=transformer) 
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=64)

In [9]:
epochs = 100
model = model.to(device)
earlystopper = EarlyStopper()
# mtcnn unable to detect gan/diffusion faces
#data = data.to(device)  # Move data to GPU if available
# half precision
# A6000 at desktop
for epoch in range(epochs): 
    running_loss = 0
    correct = 0
    total = 0
    model.train()
    for data, labels in train_loader:
        data = data.squeeze(0)
        labels = labels.squeeze(0)
        labels = labels.to(device).float()
        optimizer.zero_grad()  # clear previous gradients

        data = data.to(device).float()
        outputs = model(data)
        losses = loss_fn(outputs, labels)
        running_loss += losses.item()  # accumulate loss
        predicted = (outputs >= 0.5).float() # calculate if label is 0 or 1
        losses.backward()
        correct += (predicted == labels).sum().item() 
        total += labels.size(0)
        print(losses)

    average_train_loss = running_loss / total
    average_train_accuracy = correct / total
    train_loss_history.append(average_train_loss)
    train_accuracy_history.append(average_train_accuracy)

    model.eval()
    running_loss = 0
    correct = 0
    total = 0 
    
    print(f"Epoch [{epoch}/{epochs}], Train Loss: {average_train_loss:.4f}, Train Accuracy: {average_train_accuracy:.4f}")
    #if earlystopper.early_stop(average_val_loss, model):
    #    break 
torch.save(model.state_dict(), 'model.pth')

tensor(0.7364, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7113, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7074, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7565, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7455, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7384, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7352, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7320, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7206, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7156, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7014, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7360, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7049, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7217, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor

KeyboardInterrupt: 

In [11]:
torch.save(model.state_dict(), 'model.pth')

In [9]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))